In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype('float32')/255.0
x_test  =  x_test.reshape(-1, 28*28).astype('float32')/255.0

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
class MyModel(keras.Model):
  def __init__(self, num_classes=10):
    super(MyModel, self).__init__()
    self.dense1 = layers.Dense(64)
    self.dense2 = layers.Dense(num_classes)
  
  def call(self, input_tensor):
    x = keras.activations.relu(self.dense1(input_tensor))
    return self.dense2(x)
  

model = MyModel()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_train, y_train, batch_size=32, verbose=2)

Epoch 1/2
1875/1875 - 2s - loss: 0.3078 - accuracy: 0.9133
Epoch 2/2
1875/1875 - 2s - loss: 0.1521 - accuracy: 0.9554
1875/1875 - 1s - loss: 0.1104 - accuracy: 0.9687


[0.11041275411844254, 0.9686833620071411]

Layer

In [9]:
class Dense(layers.Layer):
  def __init__(self, units, input_dim):
    super(Dense, self).__init__()
    # name is neccesay if you want to save model
    self.w = self.add_weight(
        name='w',
        shape=(input_dim, units),
        initializer='random_normal',
        trainable=True,
    )

    self.b = self.add_weight(
        name='b', shape=(units,), initializer='zeros', trainable=True
    )

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

In [10]:
class MyModel(keras.Model):
  def __init__(self, num_classes=10):
    super(MyModel, self).__init__()
    self.dense1 = Dense(64, 784)
    self.dense2 = Dense(10, 64)
  
  def call(self, input_tensor):
    x = keras.activations.relu(self.dense1(input_tensor))
    return self.dense2(x)
  

model = MyModel()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_train, y_train, batch_size=32, verbose=2)

Epoch 1/2
1875/1875 - 2s - loss: 0.3349 - accuracy: 0.9093
Epoch 2/2
1875/1875 - 2s - loss: 0.1568 - accuracy: 0.9539
1875/1875 - 1s - loss: 0.1242 - accuracy: 0.9630


[0.12418439984321594, 0.9629666805267334]

In [11]:
print(model.summary())

Model: "my_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              multiple                  50240     
_________________________________________________________________
dense_4 (Dense)              multiple                  650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
None


Lazy layer

In [15]:
class MyReLU(layers.Layer):
  def __init__(self):
    super(MyReLU, self).__init__()
  
  def call(self, x):
    return tf.math.maximum(x, 0)

In [16]:
class Dense(layers.Layer):
  def __init__(self, units):
    super(Dense, self).__init__()
    # name is neccesay if you want to save model
    self.units=units

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b
  
  def build(self, input_shape):
    input_dim = input_shape[-1]
    
    self.w = self.add_weight(
        name='w',
        shape=(input_dim, self.units),
        initializer='random_normal',
        trainable=True,
    )

    self.b = self.add_weight(
        name='b', shape=(self.units,), initializer='zeros', trainable=True
    )

In [17]:
class MyModel(keras.Model):
  def __init__(self, num_classes=10):
    super(MyModel, self).__init__()
    self.dense1 = Dense(64)
    self.dense2 = Dense(10)
    self.relu = MyReLU()
  
  def call(self, input_tensor):
    x = self.relu(self.dense1(input_tensor))
    return self.dense2(x)
  

model = MyModel()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_train, y_train, batch_size=32, verbose=2)

Epoch 1/2
1875/1875 - 2s - loss: 0.3418 - accuracy: 0.9055
Epoch 2/2
1875/1875 - 2s - loss: 0.1655 - accuracy: 0.9515
1875/1875 - 1s - loss: 0.1269 - accuracy: 0.9626


[0.12685681879520416, 0.9625999927520752]

In [18]:
print(model.summary())

Model: "my_model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              multiple                  50240     
_________________________________________________________________
dense_8 (Dense)              multiple                  650       
_________________________________________________________________
my_re_lu (MyReLU)            multiple                  0         
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
None
